In [1]:
import os
import pandas as pd

PROJECT_ROOT = "/workspace/ship-ai"  # 너 환경에 맞게
PROCESSED_DIR = os.path.join(PROJECT_ROOT, "data", "processed")

PATH_FINAL = os.path.join(PROCESSED_DIR, "final_master_table_v2.csv")
PATH_PRICE = os.path.join(PROCESSED_DIR, "master.csv")
PATH_OUT   = os.path.join(PROCESSED_DIR, "master_table_denorm.csv")

print("[INFO] final_master_table_v2:", PATH_FINAL)
print("[INFO] master.csv:", PATH_PRICE)
print("[INFO] output:", PATH_OUT)

# 1) final_master_table_v2 로드 (펀더멘털/지수 쪽)
df_feat = pd.read_csv(
    PATH_FINAL,
    parse_dates=["date"],
    dtype={"ticker": str}
)

# ticker zero-fill
df_feat["ticker"] = df_feat["ticker"].str.zfill(6)

print("[feat] columns:", df_feat.columns.tolist()[:20])

# 우리가 쓸 컬럼만 추리기
cols_from_feat = [
    "date",
    "ticker",
    "roe",
    "real_debt_ratio",
    "bdi_proxy",
    "wti",
    "newbuild_proxy_2015_100",
]

missing_in_feat = [c for c in cols_from_feat if c not in df_feat.columns]
if missing_in_feat:
    print("[WARN] df_feat에서 빠진 컬럼:", missing_in_feat)

df_feat_small = df_feat[ [c for c in cols_from_feat if c in df_feat.columns] ].copy()

# rename: real_debt_ratio -> debt_ratio, newbuild_proxy_2015_100 -> newbuild_proxy
df_feat_small = df_feat_small.rename(columns={
    "real_debt_ratio": "debt_ratio",
    "newbuild_proxy_2015_100": "newbuild_proxy",
})

print("[feat_small] head:")
print(df_feat_small.head())

# 2) master.csv 로드 (가격 원본)
df_price = pd.read_csv(
    PATH_PRICE,
    parse_dates=["date"],
    dtype={"ticker": str}
)

df_price["ticker"] = df_price["ticker"].str.zfill(6)

print("[price] columns:", df_price.columns.tolist()[:20])

# trading_volum 오타 가능성 처리
if "trading_volum" in df_price.columns and "trading_volume" not in df_price.columns:
    df_price = df_price.rename(columns={"trading_volum": "trading_volume"})

cols_from_price = [
    "date",
    "ticker",
    "open",
    "high",
    "low",
    "close",
    "trading_volume",
]

missing_in_price = [c for c in cols_from_price if c not in df_price.columns]
if missing_in_price:
    print("[WARN] df_price에서 빠진 컬럼:", missing_in_price)

df_price_small = df_price[ [c for c in cols_from_price if c in df_price.columns] ].copy()

print("[price_small] head:")
print(df_price_small.head())

# 3) 두 테이블 merge (date, ticker 기준 inner join)
df_denorm = pd.merge(
    df_price_small,
    df_feat_small,
    on=["date", "ticker"],
    how="inner"
)

# 4) 정렬 및 컬럼 순서 정리
col_order = [
    "date",
    "ticker",
    "open", "high", "low", "close", "trading_volume",
    "roe", "debt_ratio",
    "bdi_proxy", "newbuild_proxy", "wti",
]

# 실제 있는 컬럼만 순서대로 사용
col_order_final = [c for c in col_order if c in df_denorm.columns]
df_denorm = df_denorm[col_order_final].sort_values(["ticker", "date"]).reset_index(drop=True)

print("[denorm] shape:", df_denorm.shape)
print("[denorm] head:")
print(df_denorm.head())

# 5) 저장
df_denorm.to_csv(PATH_OUT, index=False)
print("[SAVE] master_table_denorm.csv saved to:", PATH_OUT)
print("[denorm] tickers:", df_denorm["ticker"].unique()[:10], "...")


[INFO] final_master_table_v2: /workspace/ship-ai/data/processed/final_master_table_v2.csv
[INFO] master.csv: /workspace/ship-ai/data/processed/master.csv
[INFO] output: /workspace/ship-ai/data/processed/master_table_denorm.csv
[feat] columns: ['date', 'ticker', 'close_log', 'ret_1d', 'trading_volume_log', 'roe', 'real_debt_ratio', 'new_order_event_impulse', 'new_order_count_stair', 'bdi_proxy', 'wti', 'newbuild_proxy_2015_100', 'imo_event_impulse', 'imo_event_decay']
[feat_small] head:
        date  ticker       roe  debt_ratio  bdi_proxy        wti  \
0 2019-05-15  010140 -1.539182  119.929095     11.800  62.020000   
1 2019-05-16  010140 -1.539182  119.929095     11.835  62.869999   
2 2019-05-17  010140 -1.539182  119.929095     12.025  62.759998   
3 2019-05-20  010140 -1.539182  119.929095     11.835  63.099998   
4 2019-05-21  010140 -1.539182  119.929095     11.965  62.990002   

   newbuild_proxy  
0      102.848735  
1      102.848735  
2      102.848735  
3      102.848735  


## 회의 후 수정

In [3]:
import os
import pandas as pd

PROJECT_ROOT = "/workspace/ship-ai"  # 너 환경에 맞게
PROCESSED_DIR = os.path.join(PROJECT_ROOT, "data", "processed")

PATH_FINAL = os.path.join(PROCESSED_DIR, "final_master_table_v2.csv")
PATH_PRICE = os.path.join(PROCESSED_DIR, "master.csv")
PATH_OUT   = os.path.join(PROCESSED_DIR, "master_table_denorm.csv")

print("[INFO] final_master_table_v2:", PATH_FINAL)
print("[INFO] master.csv:", PATH_PRICE)
print("[INFO] output:", PATH_OUT)

# 1) final_master_table_v2 로드 (펀더멘털/지수 쪽)
df_feat = pd.read_csv(
    PATH_FINAL,
    parse_dates=["date"],
    dtype={"ticker": str}
)

# ticker zero-fill
df_feat["ticker"] = df_feat["ticker"].str.zfill(6)

print("[feat] columns:", df_feat.columns.tolist()[:20])

# 우리가 쓸 컬럼만 추리기
cols_from_feat = [
    "date",
    "ticker",
    "ret_1d",
    "roe",
    "real_debt_ratio",
    "bdi_proxy",
    "wti",
    "newbuild_proxy_2015_100",
]

missing_in_feat = [c for c in cols_from_feat if c not in df_feat.columns]
if missing_in_feat:
    print("[WARN] df_feat에서 빠진 컬럼:", missing_in_feat)

df_feat_small = df_feat[ [c for c in cols_from_feat if c in df_feat.columns] ].copy()

# rename: real_debt_ratio -> debt_ratio, newbuild_proxy_2015_100 -> newbuild_proxy
df_feat_small = df_feat_small.rename(columns={
    "real_debt_ratio": "debt_ratio",
    "newbuild_proxy_2015_100": "newbuild_proxy",
})

print("[feat_small] head:")
print(df_feat_small.head())

# 2) master.csv 로드 (가격 원본)
df_price = pd.read_csv(
    PATH_PRICE,
    parse_dates=["date"],
    dtype={"ticker": str}
)

df_price["ticker"] = df_price["ticker"].str.zfill(6)

print("[price] columns:", df_price.columns.tolist()[:20])

cols_from_price = [
    "date",
    "ticker",
    "open",
    "high",
    "low",
    "close",
    "trading_volume",
]

missing_in_price = [c for c in cols_from_price if c not in df_price.columns]
if missing_in_price:
    print("[WARN] df_price에서 빠진 컬럼:", missing_in_price)

df_price_small = df_price[ [c for c in cols_from_price if c in df_price.columns] ].copy()

print("[price_small] head:")
print(df_price_small.head())

# 3) 두 테이블 merge (date, ticker 기준 inner join)
df_denorm = pd.merge(
    df_price_small,
    df_feat_small,
    on=["date", "ticker"],
    how="inner"
)

# 4) 정렬 및 컬럼 순서 정리
col_order = [
    "date",
    "ticker",
    "open", "high", "low", "close","ret_1d", "trading_volume",
    "roe", "debt_ratio",
    "bdi_proxy", "newbuild_proxy", "wti",
]

# 실제 있는 컬럼만 순서대로 사용
col_order_final = [c for c in col_order if c in df_denorm.columns]
df_denorm = df_denorm[col_order_final].sort_values(["ticker", "date"]).reset_index(drop=True)

print("[denorm] shape:", df_denorm.shape)
print("[denorm] head:")
print(df_denorm.head())

# 5) 저장
df_denorm.to_csv(PATH_OUT, index=False)
print("[SAVE] master_table_denorm.csv saved to:", PATH_OUT)
print("[denorm] tickers:", df_denorm["ticker"].unique()[:10], "...")


[INFO] final_master_table_v2: /workspace/ship-ai/data/processed/final_master_table_v2.csv
[INFO] master.csv: /workspace/ship-ai/data/processed/master.csv
[INFO] output: /workspace/ship-ai/data/processed/master_table_denorm.csv
[feat] columns: ['date', 'ticker', 'close_log', 'ret_1d', 'trading_volume_log', 'roe', 'real_debt_ratio', 'new_order_event_impulse', 'new_order_count_stair', 'bdi_proxy', 'wti', 'newbuild_proxy_2015_100', 'imo_event_impulse', 'imo_event_decay']
[feat_small] head:
        date  ticker    ret_1d       roe  debt_ratio  bdi_proxy        wti  \
0 2019-05-15  010140  0.016545 -1.539182  119.929095     11.800  62.020000   
1 2019-05-16  010140 -0.021712 -1.539182  119.929095     11.835  62.869999   
2 2019-05-17  010140 -0.015664 -1.539182  119.929095     12.025  62.759998   
3 2019-05-20  010140 -0.012963 -1.539182  119.929095     11.835  63.099998   
4 2019-05-21  010140  0.010467 -1.539182  119.929095     11.965  62.990002   

   newbuild_proxy  
0      102.848735  
